#Planning under uncertainty in temporal availability constraints considering only one type of uncertainty (Genetic Algoritm): Robots Context.

In [ ]:
import random

# Number of data rows you want to generate
num_filas = 20

# Output file name
nombre_archivo = "pacientes2.txt"

# Open the file in write mode
with open(nombre_archivo, "w") as archivo:
    for i in range(1, num_filas + 1):
        # Generate random values
        campo1 = i
        campo2 = random.randint(100, 200)
        campo3 = random.randint(8, 14)
        campo4 = random.randint(9, 18)
        campo5 = 0

        # Write the line
        linea = f"{campo1}, {campo2}, {campo3}, {campo4}, {campo5}\n"
        archivo.write(linea)

print(f"Se ha generado el archivo '{nombre_archivo}' con datos aleatorios.")


## Classes required for the algorithm's operation

In [ ]:
# Class representing the patient
class Patient:
    def __init__(self, id, distance, available_time_start, available_time_end, has_food):
        self.id = id  # Patient identifyer
        self.distance_room = distance  # Distance from the starting point to the robot's room
        self.available_time_start = available_time_start  # Available start time for appointment
        self.available_time_end = available_time_end  # Available end time for appointment
        self.has_food = has_food  # The patient has food


    #Calculate and return the time required to deliver the meal to a patient (standard speed 5 km/h
    def get_attending_time(self):
        required_time=0

        if self.has_food == False:
          required_time = 2*(self.distance_room/80) #Calculate and return the time required to deliver the meal to a patient, assuming a standard speed of 5 km/h (approximately 80 m/min, so divide by 80).

        return required_time

    def __str__(self):
      return f"Paciente {self.id}"

In [ ]:
# Class representing a robot
class Robot:
    def __init__(self, id):
        self.id = id
        self.periodos_ocupado = []
        self.current_charge = 10
        self.velocidad = 5 #5km/h

    def set_estado(self, hora_inicio, hora_fin):
        self.periodos_ocupado.append((hora_inicio, hora_fin))

    def eliminar_estado(self, hora_inicio, hora_fin):
        self.periodos_ocupado = [(inicio, fin) for inicio, fin in self.periodos_ocupado if (inicio, fin) != (hora_inicio, hora_fin)]

    def liberar_robot(self):
        self.periodos_ocupado = []

    def esta_ocupado(self, hora_inicio, hora_fin):
        for inicio, fin in self.periodos_ocupado:
            if inicio <= hora_fin and fin >= hora_inicio:
                return True
        return False

    def tiene_horas_ocupadas(self):
        return bool(self.periodos_ocupado)

In [ ]:
# Class representing a element of solution list
class item_planlist:
    def __init__(self, patient, begin_time, end_time, robot):
        self.patient = patient
        self.tiempo_inicio = begin_time
        self.tiempo_fin = end_time
        self.robot = robot

In [ ]:
import random
import numpy as np

In [ ]:
# Calculates the timespan of a solution
def calcular_timespan(solucion):
    tiempo_inicio_mas_temprano = min(item.tiempo_inicio for item in solucion)
    tiempo_fin_mas_tardio = max(item.tiempo_fin for item in solucion)
    return tiempo_fin_mas_tardio - tiempo_inicio_mas_temprano

## Basic methods of the genetic algorithm

In [ ]:
# Method that generates the initial population
def generar_poblacion_inicial(pacientes, robots, population_size):
    #Set of solutions
    poblacion = []
    for _ in range(population_size):

        solucion = []

        #Release robots
        for i in range(len(robots)):
          robots[i].liberar_robot()


        for paciente in pacientes:
          #Check if the patient needs food
          if paciente.get_attending_time() > 0:

            tiempo_inicio = random.uniform(paciente.available_time_start, paciente.available_time_end-0.5)
            tiempo_fin = random.uniform(tiempo_inicio+0.5,tiempo_inicio+paciente.get_attending_time())
            tiempo_fin = min(tiempo_fin, paciente.available_time_end)  # Asegurar que el tiempo de carga no exceda el tiempo disponible

            rebot = None

            #Assign robot
            for i in range(len(robots)):
              if(robots[i].esta_ocupado(tiempo_inicio, tiempo_fin) == False):
                robot = robots[i]
                robots[i].set_estado(tiempo_inicio, tiempo_fin)
                break

            if(robot != None):
              it = item_planlist(paciente,tiempo_inicio,tiempo_fin,robot)
              solucion.append(it)

        #dd solution to poputation
        poblacion.append(solucion)
    return poblacion

In [ ]:
# Method that evaluates the solutions
def evaluar_poblacion(poblacion, hora_llegada_varianza, num_muestras=50):
    evaluaciones = []

    #Para cada solucición en la población
    for solucion in poblacion:
        tiempo_total = 0
        timespan_ini = 0
        timespan_fin = 0
        acum_timespan_fin = 0
        acum_timespan_ini = 0

        # For each solution in the population
        for i, item in enumerate(solucion):
            # Generate N random samples for the initial time before the mean arrival time, hour_arrival_mean[i]
            muestras_llegada = np.random.normal(item.patient.available_time_start, hora_llegada_varianza, num_muestras)
            tiempo_muestras = 0


            # The same for end times
            muestras_salida = np.random.normal(item.patient.available_time_end, hora_llegada_varianza, num_muestras)

            # Calculate the sum
            suma = sum(muestras_salida)

            # Calculate number of elements
            cantidad_elementos = len(muestras_salida)

            # Calcular la media
            media_salida = suma / cantidad_elementos


            # Check that no sample is outside the availability
            for muestra_llegada in muestras_llegada:
                tiempo_inicio = muestra_llegada
                tiempo_fin = min(muestra_llegada + item.patient.get_attending_time(), media_salida)
                tiempo_carga = tiempo_fin - tiempo_inicio
                tiempo_muestras += tiempo_carga

                if i == (len(solucion)-1):
                  acum_timespan_fin += tiempo_fin

                if i == 0:
                  acum_timespan_ini = tiempo_inicio

            tiempo_total += tiempo_muestras / num_muestras
            timespan_fin = acum_timespan_fin / num_muestras
            timespan_ini = acum_timespan_ini / num_muestras


        timespan = timespan_fin - timespan_ini
        evaluaciones.append((solucion, tiempo_total, timespan))

    return evaluaciones



# method that selects the parents or best solutions
def seleccionar_padres(evaluaciones, num_padres):
    evaluaciones.sort(key=lambda x: (x[1], x[2]))
    padres = []
    for i in range(num_padres):
        padres.append(evaluaciones[i][0])

    return padres

In [ ]:
# Crossover of parents
def cruzar_padres(padre1, padre2):
    hijo = []

    # Release robots
    for i in range(len(robots)):
        robots[i].liberar_robot()

    for i in range(len(padre1)):

        tiempo_inicio_hijo = min(padre1[i].tiempo_inicio, padre2[i].tiempo_inicio)
        tiempo_fin_hijo = max(padre1[i].tiempo_fin, padre2[i].tiempo_fin)

        robot = None

        #Assign robot
        for j in range(len(robots)):
            if(robots[j].esta_ocupado(tiempo_inicio_hijo, tiempo_fin_hijo) == False):
              robot = robots[j]
              robots[j].set_estado(tiempo_inicio_hijo, tiempo_fin_hijo)
              break

        item = item_planlist(padre1[i].patient, tiempo_inicio_hijo, tiempo_fin_hijo, robot)

        if (robot == None):
              item = item_planlist(padre1[i].patient, padre1[i].tiempo_inicio, padre1[i].tiempo_fin, padre1[i].robot)

        hijo.append(item)

    return hijo

In [ ]:
#Introduce mutations
def mutar_hijo(hijo, mutation_rate):

    for i in range(len(robots)):
      robots[i].liberar_robot()

    for i in range(len(hijo)):
        if random.uniform(0, 1) < mutation_rate:
            nuevo_tiempo_inicio = random.uniform(hijo[i].patient.available_time_start, hijo[i].patient.available_time_end - 0.5)
            nuevo_tiempo_fin = nuevo_tiempo_inicio + hijo[i].patient.get_attending_time()
            nuevo_tiempo_fin = min(nuevo_tiempo_fin, hijo[i].patient.available_time_end)

            if nuevo_tiempo_fin == nuevo_tiempo_inicio:
                nuevo_tiempo_fin = nuevo_tiempo_inicio + 0.5


            for j in range(len(robots)):
              if(robots[j].esta_ocupado(nuevo_tiempo_inicio, nuevo_tiempo_fin) == False):
                robots[j].set_estado(nuevo_tiempo_inicio, nuevo_tiempo_fin)
                hijo[i].tiempo_inicio = nuevo_tiempo_inicio
                hijo[i].tiempo_fin = nuevo_tiempo_fin
                hijo[i].robot = robots[j]
                break

    return hijo


In [ ]:
# Parameters
population_size = 10
num_generations = 50
num_padres = 3
mutation_rate = 0.2

# Generate robots list
robots = [Robot(1), Robot(2), Robot(3), Robot(2), Robot(5), Robot(6)]

#, Robot(3), Robot(4), Robot(5), Robot(6), Robot(7), Robot(8), Robot(9), Robot(10), Robot(11), Robot(12)

'''
# Generar ejemplos de pacientes
paciente1 = Patient(1, 80, 9, 10, 0)
paciente2 = Patient(2, 200, 8, 9, 0)
paciente3 = Patient(3, 150, 12, 13, 0)
paciente4 = Patient(4, 200, 12, 13, 0)
paciente5 = Patient(5, 100, 14, 17, 0)
paciente6 = Patient(6, 200, 13, 18, 0)

pacientes = [paciente1, paciente2, paciente3, paciente4, paciente5, paciente6]'''

# Inicialize patients from file
def create_patients_from_file(file_name):
    pacientes = []
    with open(file_name, 'r') as file:
        lines = file.readlines()
        for line in lines:
            data = line.strip().split(', ')
            if len(data) == 5:
                id, distance, available_time_start, available_time_end, has_food = map(float, data)
                paciente = Patient(id, distance, available_time_start, available_time_end, has_food)
                pacientes.append(paciente)
    return pacientes

file_name = 'pacientes2.txt'
pacientes = create_patients_from_file(file_name)

hora_llegada_varianza = 1

#Generate initial population
poblacion = generar_poblacion_inicial(pacientes, robots, population_size)

print("POBLACION")
for sol in poblacion:
  for it in sol:
    print("identificador paciente: "+ str(it.patient.id))
    print("inicio "+str(it.tiempo_inicio))
    print("fin "+str(it.tiempo_fin))
    print("robot "+str(it.robot.id))
    print("")

  print("-------------------------------------------------")

# Run algorithm
for generation in range(num_generations):
    evaluaciones = evaluar_poblacion(poblacion, hora_llegada_varianza)
    padres = seleccionar_padres(evaluaciones, num_padres)

    nueva_poblacion = []
    for i in range(population_size):
        padre1 = random.choice(padres)
        padre2 = random.choice(padres)
        hijo = cruzar_padres(padre1, padre2)
        hijo_mutado = mutar_hijo(hijo, mutation_rate)
        nueva_poblacion.append(hijo_mutado)

    poblacion = nueva_poblacion

# Get the best solution
evaluaciones = evaluar_poblacion(poblacion, hora_llegada_varianza)
evaluaciones.sort(key=lambda x: (x[1], x[2]))
mejor_sol = evaluaciones[0]



# Imprimir la mejor solución encontrada
print("Tiempo total: " + str(mejor_sol[1]))
print("Timespan " + str(mejor_sol[2]) )
print("Planificación:")

for item in mejor_sol[0]:
  print(item.patient.id)
  print(item.tiempo_inicio)
  print(item.tiempo_fin)
  print(item.robot.id)
  print("")



#STATISTICS
'''file_name2 = 'pacientesprueba.txt'
pacientes = create_patients_from_file(file_name2)


i=0
cont=0
for item in mejor_sol[0]:
  if item.patient.id == pacientes[i].id:
    if item.tiempo_inicio<pacientes[i].available_time_start:
      cont=cont+1

  i=i+1

print("Número de acompañantes con retraso")
print(cont)'''


#Planning under uncertainty in temporal availability constraints considering two types of uncertainties (Genetic Algoritm): Robots Context.

In [ ]:
import random

# Number of data rows you want to generate
num_filas = 20

# Output file name
nombre_archivo = "pacientes2.txt"

# Open the file in write mode
with open(nombre_archivo, "w") as archivo:
    for i in range(1, num_filas + 1):
        # Generate random values
        campo1 = i
        campo2 = random.randint(100, 200)
        campo3 = random.randint(8, 14)
        campo4 = random.randint(9, 18)
        campo5 = 0

        # Write the line
        linea = f"{campo1}, {campo2}, {campo3}, {campo4}, {campo5}\n"
        archivo.write(linea)

print(f"Se ha generado el archivo '{nombre_archivo}' con datos aleatorios.")


## Classes required for the algorithm's operation

In [ ]:
# Class representing the patient
class Patient:
    def __init__(self, id, distance, available_time_start, available_time_end, has_food):
        self.id = id  # Patient identifyer
        self.distance_room = distance  # Distance from the starting point to the robot's room
        self.available_time_start = available_time_start  # Available start time for appointment
        self.available_time_end = available_time_end  # Available end time for appointment
        self.has_food = has_food  # The patient has food


    #Calculate and return the time required to deliver the meal to a patient (standard speed 5 km/h
    def get_attending_time(self):
        required_time=0

        if self.has_food == False:
          required_time = 2*(self.distance_room/80) #Calculate and return the time required to deliver the meal to a patient, assuming a standard speed of 5 km/h (approximately 80 m/min, so divide by 80).

        return required_time

    def __str__(self):
      return f"Paciente {self.id}"

In [ ]:
# Class representing a robot
class Robot:
    def __init__(self, id):
        self.id = id
        self.periodos_ocupado = []
        self.current_charge = 10
        self.velocidad = 5 #5km/h

    def set_estado(self, hora_inicio, hora_fin):
        self.periodos_ocupado.append((hora_inicio, hora_fin))

    def eliminar_estado(self, hora_inicio, hora_fin):
        self.periodos_ocupado = [(inicio, fin) for inicio, fin in self.periodos_ocupado if (inicio, fin) != (hora_inicio, hora_fin)]

    def liberar_robot(self):
        self.periodos_ocupado = []

    def esta_ocupado(self, hora_inicio, hora_fin):
        for inicio, fin in self.periodos_ocupado:
            if inicio <= hora_fin and fin >= hora_inicio:
                return True
        return False

    def tiene_horas_ocupadas(self):
        return bool(self.periodos_ocupado)

In [ ]:
# Class representing a element of solution list
class item_planlist:
    def __init__(self, patient, begin_time, end_time, robot):
        self.patient = patient
        self.tiempo_inicio = begin_time
        self.tiempo_fin = end_time
        self.robot = robot

In [ ]:
import random
import numpy as np

In [ ]:
# Calculates the timespan of a solution
def calcular_timespan(solucion):
    tiempo_inicio_mas_temprano = min(item.tiempo_inicio for item in solucion)
    tiempo_fin_mas_tardio = max(item.tiempo_fin for item in solucion)
    return tiempo_fin_mas_tardio - tiempo_inicio_mas_temprano

## Basic methods of the genetic algorithm

In [ ]:
# Method that generates the initial population
def generar_poblacion_inicial(pacientes, robots, population_size):
    #Set of solutions
    poblacion = []
    for _ in range(population_size):

        solucion = []

        #Release robots
        for i in range(len(robots)):
          robots[i].liberar_robot()


        for paciente in pacientes:
          #Check if the patient needs food
          if paciente.get_attending_time() > 0:

            tiempo_inicio = random.uniform(paciente.available_time_start, paciente.available_time_end-0.5)
            tiempo_fin = random.uniform(tiempo_inicio+0.5,tiempo_inicio+paciente.get_attending_time())
            tiempo_fin = min(tiempo_fin, paciente.available_time_end)  # Asegurar que el tiempo de carga no exceda el tiempo disponible

            rebot = None

             #----------------------------------------------------------------------------------------------------

            for j in range(len(robots)):

              #Robots broken down in the first time slot
              for i in range (0,8):
                num = random.uniform(0, 1)
                t_ini = i
                t_fin = i+1
                if num < 0.01:
                  robots[j].set_estado(t_ini, t_fin)

              #Robots broken down in the second time slot
              for i in range (8,14):
                num = random.uniform(0, 1)
                t_ini = i
                t_fin = i+1
                if num < 0.02:
                  robots[j].set_estado(t_ini, t_fin)

              #Robots broken down in the third time slot
              for i in range (14,19):
                num = random.uniform(0, 1)
                t_ini = i
                t_fin = i+1
                if num < 0.05:
                  robots[j].set_estado(t_ini, t_fin)

              #Robots broken down in the fourth time slot
              for i in range (19,23):
                num = random.uniform(0, 1)
                t_ini = i
                t_fin = i+1
                if num < 0.01:
                  robots[j].set_estado(t_ini, t_fin)

            #----------------------------------------------------------------------------------------------------

            #Assign robot
            for i in range(len(robots)):
              if(robots[i].esta_ocupado(tiempo_inicio, tiempo_fin) == False):
                robot = robots[i]
                robots[i].set_estado(tiempo_inicio, tiempo_fin)
                break

            if(robot != None):
              it = item_planlist(paciente,tiempo_inicio,tiempo_fin,robot)
              solucion.append(it)

        #Add solution to poputation
        poblacion.append(solucion)
    return poblacion

In [ ]:
# Method that evaluates the solutions
def evaluar_poblacion(poblacion,robots, hora_llegada_varianza, num_muestras=50):
    evaluaciones = []

    #Para cada solucición en la población
    for solucion in poblacion:
        tiempo_total = 0
        timespan_ini = 0
        timespan_fin = 0
        acum_timespan_fin = 0
        acum_timespan_ini = 0

        #Establecer la lista de robots que se estropean
        for j in range(len(robots)):

              #Robots broken down in the first time slot
              for i in range (0,8):
                num = random.uniform(0, 1)
                t_ini = i
                t_fin = i+1
                if num < 0.01:
                  robots[j].set_estado(t_ini, t_fin)

              #Robots broken down in the second time slot
              for i in range (8,14):
                num = random.uniform(0, 1)
                t_ini = i
                t_fin = i+1
                if num < 0.02:
                  robots[j].set_estado(t_ini, t_fin)

              #Robots broken down in the third time slot
              for i in range (14,19):
                num = random.uniform(0, 1)
                t_ini = i
                t_fin = i+1
                if num < 0.05:
                  robots[j].set_estado(t_ini, t_fin)

              #Robots broken down in the fourth time slot
              for i in range (19,23):
                num = random.uniform(0, 1)
                t_ini = i
                t_fin = i+1
                if num < 0.01:
                  robots[j].set_estado(t_ini, t_fin)

        # For each solution in the population
        for i, item in enumerate(solucion):
            # Generate N random samples for the initial time before the mean arrival time, hour_arrival_mean[i]
            muestras_llegada = np.random.normal(item.patient.available_time_start, hora_llegada_varianza, num_muestras)
            tiempo_muestras = 0


            # The same for end times
            muestras_salida = np.random.normal(item.patient.available_time_end, hora_llegada_varianza, num_muestras)

            # Calculate the sum
            suma = sum(muestras_salida)

            # Calculate number of elements
            cantidad_elementos = len(muestras_salida)

            # Calcular la media
            media_salida = suma / cantidad_elementos


            # Check that no sample is outside the availability
            for muestra_llegada in muestras_llegada:
                tiempo_inicio = muestra_llegada
                tiempo_fin = min(muestra_llegada + item.patient.get_attending_time(), media_salida)
                tiempo_carga = tiempo_fin - tiempo_inicio
                tiempo_muestras += tiempo_carga

                if i == (len(solucion)-1):
                  acum_timespan_fin += tiempo_fin

                if i == 0:
                  acum_timespan_ini = tiempo_inicio

            tiempo_total += tiempo_muestras / num_muestras
            timespan_fin = acum_timespan_fin / num_muestras
            timespan_ini = acum_timespan_ini / num_muestras


        timespan = timespan_fin - timespan_ini
        evaluaciones.append((solucion, tiempo_total, timespan))

    return evaluaciones



# method that selects the parents or best solutions
def seleccionar_padres(evaluaciones, num_padres):
    evaluaciones.sort(key=lambda x: (x[1], x[2]))
    padres = []
    for i in range(num_padres):
        padres.append(evaluaciones[i][0])

    return padres

In [ ]:
# Crossover of parents
def cruzar_padres(padre1, padre2):
    hijo = []

    # Release robots
    for i in range(len(robots)):
        robots[i].liberar_robot()

    for i in range(len(padre1)):

        tiempo_inicio_hijo = min(padre1[i].tiempo_inicio, padre2[i].tiempo_inicio)
        tiempo_fin_hijo = max(padre1[i].tiempo_fin, padre2[i].tiempo_fin)

        robot = None

        #Assign robot
        for j in range(len(robots)):
            if(robots[j].esta_ocupado(tiempo_inicio_hijo, tiempo_fin_hijo) == False):
              robot = robots[j]
              robots[j].set_estado(tiempo_inicio_hijo, tiempo_fin_hijo)
              break

        item = item_planlist(padre1[i].patient, tiempo_inicio_hijo, tiempo_fin_hijo, robot)

        if (robot == None):
              item = item_planlist(padre1[i].patient, padre1[i].tiempo_inicio, padre1[i].tiempo_fin, padre1[i].robot)

        hijo.append(item)

    return hijo

In [ ]:
#Introduce mutations
def mutar_hijo(hijo, mutation_rate):

    for i in range(len(robots)):
      robots[i].liberar_robot()

    for i in range(len(hijo)):
        if random.uniform(0, 1) < mutation_rate:
            nuevo_tiempo_inicio = random.uniform(hijo[i].patient.available_time_start, hijo[i].patient.available_time_end - 0.5)
            nuevo_tiempo_fin = nuevo_tiempo_inicio + hijo[i].patient.get_attending_time()
            nuevo_tiempo_fin = min(nuevo_tiempo_fin, hijo[i].patient.available_time_end)

            if nuevo_tiempo_fin == nuevo_tiempo_inicio:
                nuevo_tiempo_fin = nuevo_tiempo_inicio + 0.5


            for j in range(len(robots)):
              if(robots[j].esta_ocupado(nuevo_tiempo_inicio, nuevo_tiempo_fin) == False):
                robots[j].set_estado(nuevo_tiempo_inicio, nuevo_tiempo_fin)
                hijo[i].tiempo_inicio = nuevo_tiempo_inicio
                hijo[i].tiempo_fin = nuevo_tiempo_fin
                hijo[i].robot = robots[j]
                break

    return hijo


In [ ]:
# Parameters
population_size = 10
num_generations = 50
num_padres = 3
mutation_rate = 0.2

# Generate robots list
robots = [Robot(1), Robot(2), Robot(3), Robot(2), Robot(5), Robot(6)]

#, Robot(3), Robot(4), Robot(5), Robot(6), Robot(7), Robot(8), Robot(9), Robot(10), Robot(11), Robot(12)

'''
# Generar ejemplos de pacientes
paciente1 = Patient(1, 80, 9, 10, 0)
paciente2 = Patient(2, 200, 8, 9, 0)
paciente3 = Patient(3, 150, 12, 13, 0)
paciente4 = Patient(4, 200, 12, 13, 0)
paciente5 = Patient(5, 100, 14, 17, 0)
paciente6 = Patient(6, 200, 13, 18, 0)

pacientes = [paciente1, paciente2, paciente3, paciente4, paciente5, paciente6]'''

# Inicialize patients from file
def create_patients_from_file(file_name):
    pacientes = []
    with open(file_name, 'r') as file:
        lines = file.readlines()
        for line in lines:
            data = line.strip().split(', ')
            if len(data) == 5:
                id, distance, available_time_start, available_time_end, has_food = map(float, data)
                paciente = Patient(id, distance, available_time_start, available_time_end, has_food)
                pacientes.append(paciente)
    return pacientes

file_name = 'pacientes2.txt'
pacientes = create_patients_from_file(file_name)

hora_llegada_varianza = 1

#Generate initial population
poblacion = generar_poblacion_inicial(pacientes, robots, population_size)

print("POBLACION")
for sol in poblacion:
  for it in sol:
    print("identificador paciente: "+ str(it.patient.id))
    print("inicio "+str(it.tiempo_inicio))
    print("fin "+str(it.tiempo_fin))
    print("robot "+str(it.robot.id))
    print("")

  print("-------------------------------------------------")

# Run algorithm
for generation in range(num_generations):
    evaluaciones = evaluar_poblacion(poblacion, robots, hora_llegada_varianza)
    padres = seleccionar_padres(evaluaciones, num_padres)

    nueva_poblacion = []
    for i in range(population_size):
        padre1 = random.choice(padres)
        padre2 = random.choice(padres)
        hijo = cruzar_padres(padre1, padre2)
        hijo_mutado = mutar_hijo(hijo, mutation_rate)
        nueva_poblacion.append(hijo_mutado)

    poblacion = nueva_poblacion

# Get the best solution
evaluaciones = evaluar_poblacion(poblacion, hora_llegada_varianza)
evaluaciones.sort(key=lambda x: (x[1], x[2]))
mejor_sol = evaluaciones[0]



# Imprimir la mejor solución encontrada
print("Tiempo total: " + str(mejor_sol[1]))
print("Timespan " + str(mejor_sol[2]) )
print("Planificación:")

for item in mejor_sol[0]:
  print(item.patient.id)
  print(item.tiempo_inicio)
  print(item.tiempo_fin)
  print(item.robot.id)
  print("")



#STATISTICS
'''file_name2 = 'pacientesprueba.txt'
pacientes = create_patients_from_file(file_name2)


i=0
cont=0
for item in mejor_sol[0]:
  if item.patient.id == pacientes[i].id:
    if item.tiempo_inicio<pacientes[i].available_time_start:
      cont=cont+1

  i=i+1

print("Número de acompañantes con retraso")
print(cont)'''
